In [ ]:
#pip install xgboost

In [ ]:
#pip install xgboost pandas sklearn

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice
import datetime as datetime
import glob
from matplotlib import pyplot
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
import warnings
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression as LR
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold

%matplotlib inline

plt.style.use('default')
sns.set(style="whitegrid")
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 5)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from catboost import CatBoostRegressor

In [2]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler
# the model
from sklearn.ensemble import RandomForestClassifier
# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline
# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [3]:
train_labels = pd.read_csv('train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
train_values = pd.read_csv('train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [4]:
train = train_values.merge(train_labels, on='building_id')
train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,2
669485,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
602512,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,3


In [5]:
geo_level_1_id_ohe = pd.get_dummies(train.geo_level_1_id, prefix='geo_level_1_id')
train = train.merge(geo_level_1_id_ohe, on='building_id')

In [6]:
geo_level_2_id_ohe = pd.get_dummies(train.geo_level_2_id, prefix='geo_level_2_id')
train = train.merge(geo_level_2_id_ohe, on='building_id')

In [7]:
train['geo_lvl_1_2'] = (train.geo_level_1_id.astype(int) * 10000) + (train.geo_level_2_id.astype(int))

In [8]:
train = train.drop('geo_level_1_id', axis=1)
train = train.drop('geo_level_2_id', axis=1)

In [9]:
#geo_level_3_id_ohe = pd.get_dummies(train.geo_level_3_id, prefix='geo_level_3_id')
#train = train.merge(geo_level_3_id_ohe, on='building_id')
train = train.drop('geo_level_3_id', axis=1)

In [10]:
other_floor_type_ohe = pd.get_dummies(train.other_floor_type, prefix='other_floor_type')
train = train.merge(other_floor_type_ohe, on='building_id')
train = train.drop('other_floor_type', axis=1)

In [11]:
position_ohe = pd.get_dummies(train.position, prefix='position')
train = train.merge(position_ohe, on='building_id')
train = train.drop('position', axis=1)

In [12]:
plan_configuration_ohe = pd.get_dummies(train.plan_configuration, prefix='plan_configuration')
train = train.merge(plan_configuration_ohe, on='building_id')
train = train.drop('plan_configuration', axis=1)

In [13]:
legal_ownership_status_ohe = pd.get_dummies(train.legal_ownership_status, prefix='legal_ownership_status')
train = train.merge(legal_ownership_status_ohe, on='building_id')
train = train.drop('legal_ownership_status', axis=1)

In [14]:
ground_floor_type_ohe = pd.get_dummies(train.ground_floor_type, prefix='ground_floor_type')
train = train.merge(ground_floor_type_ohe, on='building_id')
train = train.drop('ground_floor_type', axis=1)

In [15]:
foundation_type_ohe = pd.get_dummies(train.foundation_type, prefix='foundation_type')
train = train.merge(foundation_type_ohe, on='building_id')
train = train.drop('foundation_type', axis=1)

In [16]:
roof_type_ohe = pd.get_dummies(train.roof_type, prefix='roof_type')
train = train.merge(roof_type_ohe, on='building_id')
train = train.drop('roof_type', axis=1)

In [17]:
land_surface_condition_ohe = pd.get_dummies(train.land_surface_condition, prefix='land_surface_condition')
train = train.merge(land_surface_condition_ohe, on='building_id')
train = train.drop('land_surface_condition', axis=1)

train = train.drop('geo_level_1_id', axis=1)
train = train.drop('geo_level_2_id', axis=1)
train = train.drop('geo_level_3_id', axis=1)
train = train.drop('land_surface_condition', axis=1)
#train = train.drop('count_floors_pre_eq', axis=1)

train = train.drop('roof_type', axis=1)
train = train.drop('other_floor_type', axis=1)
train = train.drop('position', axis=1)
#train = train.drop('other_floor_type', axis=1)
train = train.drop('plan_configuration', axis=1)
train = train.drop('legal_ownership_status', axis=1)
train = train.drop('ground_floor_type', axis=1)
train = train.drop('foundation_type', axis=1)

In [18]:
train_value = train['damage_grade']
train_data = train.iloc[:, train.columns != 'damage_grade']

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(train_data, train_value, train_size=0.7, random_state=1234)

In [21]:
modelo = xgb.XGBClassifier(objective='reg:logistic', n_estimators=50, seed=123, learning_rate=0.3, 
                           max_depth=8, alpha=15, gamma=1.5)

modelo

XGBClassifier(alpha=15, base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=1.5,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.3, max_delta_step=None, max_depth=8,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=50, n_jobs=None, num_parallel_tree=None,
              objective='reg:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, seed=123, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)

# Modelo single

In [22]:
modelo.fit(X_train, y_train)

# Realizo las predicciones
y_pred = modelo.predict(X_train)
predicciones = [round(value) for value in y_pred]
# Evalúo las predicciones
precision_train = accuracy_score(y_train, predicciones)

# Repito el proceso con datos de evaluacion
y_pred = modelo.predict(X_validation)
predicciones = [round(value) for value in y_pred]
# Evalúo las predicciones
precision_test = accuracy_score(y_validation, predicciones)

print(modelo)
print('Precisión xgboost train/test  {0:.3f}/{1:.3f}'
      .format(precision_train, precision_test))

[20:29:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(alpha=15, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1.5, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=123, reg_alpha=15,
              reg_lambda=1, scale_pos_weight=None, seed=123, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Precisión xgboost train/test  0.720/0.715


# Modelo con random search

In [ ]:
params = {
        'gamma': [0.5, 1, 1.5, 2, 5],
        'max_depth': [3, 4, 5, 8, 10],
        'n_estimators': [5, 10, 20, 50],
        'learning_rate': [0.1, 0.3, 0.5, 0.7, 0.9],
        'alpha': [0, 5, 15, 30]
        }

folds = 3
param_comb = 500

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(modelo, param_distributions=params, 
        n_iter=param_comb, n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001)

random_search.fit(X_train, y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 125.2min


In [ ]:
predic = random_search.predict_proba(X_validation)

print(random_search.best_estimator_)

# Realizo las predicciones
y_pred = random_search.predict(X_train)
predicciones = [round(value) for value in y_pred]
# Evalúo las predicciones
precision_train = accuracy_score(y_train, predicciones)

# Repito el proceso con datos de evaluacion
y_pred = random_search.predict(X_validation)
predicciones = [round(value) for value in y_pred]
# Evalúo las predicciones
precision_test = accuracy_score(y_validation, predicciones)

print(random_search)
print('Precisión xgboost train/test  {0:.3f}/{1:.3f}'
      .format(precision_train, precision_test))

print(random_search.cv_results_)

# Modelo con Grid search

In [36]:
params = {
        'n_estimators': [5, 10, 15],
        'max_depth': [3, 5, 10]
        }

grid_search = GridSearchCV(estimator = modelo,
                        param_grid = params,
                        scoring = 'accuracy', 
                        cv = 5, 
                        verbose = 1,
                        n_jobs = -1)
grid_search.fit(X_train, y_train)

predic = grid_search.predict_proba(X_validation)

print(grid_search.best_estimator_)

# Realizo las predicciones
y_pred = grid_search.predict(X_train)
predicciones = [round(value) for value in y_pred]
# Evalúo las predicciones
precision_train = accuracy_score(y_train, predicciones)

# Repito el proceso con datos de evaluacion
y_pred = grid_search.predict(X_validation)
predicciones = [round(value) for value in y_pred]
# Evalúo las predicciones
precision_test = accuracy_score(y_validation, predicciones)

print(grid_search)
print('Precisión xgboost train/test  {0:.3f}/{1:.3f}'
      .format(precision_train, precision_test))

print(grid_search.cv_results_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  2.1min finished


[18:02:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(alpha=30, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.7, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=15, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=123, reg_alpha=30,
              reg_lambda=1, scale_pos_weight=None, seed=123, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
GridSearchCV(cv=5,
             estimator=XGBCl

# -----------------------------------------------------------

In [25]:
test = pd.read_csv('test_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [26]:
geo_level_1_id_ohe = pd.get_dummies(test.geo_level_1_id, prefix='geo_level_1_id')
test = test.merge(geo_level_1_id_ohe, on='building_id')

In [27]:
geo_level_2_id_ohe = pd.get_dummies(test.geo_level_2_id, prefix='geo_level_2_id')
test = test.merge(geo_level_2_id_ohe, on='building_id')

In [28]:
test['geo_lvl_1_2'] = (test.geo_level_1_id.astype(int) * 10000) + (test.geo_level_2_id.astype(int))

In [29]:
test = test.drop('geo_level_1_id', axis=1)
test = test.drop('geo_level_2_id', axis=1)

In [30]:
test = test.drop('geo_level_3_id', axis=1)

In [31]:
other_floor_type_ohe = pd.get_dummies(test.other_floor_type, prefix='other_floor_type')
test = test.merge(other_floor_type_ohe, on='building_id')
test = test.drop('other_floor_type', axis=1)

In [32]:
position_ohe = pd.get_dummies(test.position, prefix='position')
test = test.merge(position_ohe, on='building_id')
test = test.drop('position', axis=1)

In [33]:
plan_configuration_ohe = pd.get_dummies(test.plan_configuration, prefix='plan_configuration')
test = test.merge(plan_configuration_ohe, on='building_id')
test = test.drop('plan_configuration', axis=1)

In [34]:
legal_ownership_status_ohe = pd.get_dummies(test.legal_ownership_status, prefix='legal_ownership_status')
test = test.merge(legal_ownership_status_ohe, on='building_id')
test = test.drop('legal_ownership_status', axis=1)

In [35]:
ground_floor_type_ohe = pd.get_dummies(test.ground_floor_type, prefix='ground_floor_type')
test = test.merge(ground_floor_type_ohe, on='building_id')
test = test.drop('ground_floor_type', axis=1)

In [36]:
foundation_type_ohe = pd.get_dummies(test.foundation_type, prefix='foundation_type')
test = test.merge(foundation_type_ohe, on='building_id')
test = test.drop('foundation_type', axis=1)

In [37]:
roof_type_ohe = pd.get_dummies(test.roof_type, prefix='roof_type')
test = test.merge(roof_type_ohe, on='building_id')
test = test.drop('roof_type', axis=1)

In [38]:
land_surface_condition_ohe = pd.get_dummies(test.land_surface_condition, prefix='land_surface_condition')
test = test.merge(land_surface_condition_ohe, on='building_id')
test = test.drop('land_surface_condition', axis=1)

test = test.drop('geo_level_1_id', axis=1)
test = test.drop('geo_level_2_id', axis=1)
test = test.drop('geo_level_3_id', axis=1)
test = test.drop('land_surface_condition', axis=1)
#test = test.drop('count_floors_pre_eq', axis=1)

test = test.drop('roof_type', axis=1)
test = test.drop('other_floor_type', axis=1)
test = test.drop('position', axis=1)
#test = test.drop('other_floor_type', axis=1)
test = test.drop('plan_configuration', axis=1)
test = test.drop('legal_ownership_status', axis=1)
test = test.drop('ground_floor_type', axis=1)
test = test.drop('foundation_type', axis=1)

# Predicciones (single o random search)

predic = random_search.predict_proba(test)

In [39]:
predic = modelo.predict_proba(test)

# -----------------------------------------------------------

In [40]:
predic

array([[0.01029185, 0.23400791, 0.75570023],
       [0.04462228, 0.800049  , 0.15532872],
       [0.11174045, 0.6943553 , 0.19390422],
       ...,
       [0.04303098, 0.8044951 , 0.15247396],
       [0.09208951, 0.6346588 , 0.27325165],
       [0.3901029 , 0.5842476 , 0.02564947]], dtype=float32)

In [41]:
results = []
for line in predic:
    if line[0] > line [1] and line [0] > line [2]:
        results.append(1)
    elif line[1] > line[2]:
        results.append(2)
    else:
        results.append(3)

In [42]:
results

[3,
 2,
 2,
 2,
 3,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 3,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [43]:
damages = pd.Series(results).rename("damage_grade")

In [44]:
damages

0        3
1        2
2        2
3        2
4        3
        ..
86863    2
86864    2
86865    2
86866    2
86867    2
Name: damage_grade, Length: 86868, dtype: int64

In [45]:
ids = test.reset_index()['building_id']

In [46]:
ids

0         300051
1          99355
2         890251
3         745817
4         421793
          ...   
86863     310028
86864     663567
86865    1049160
86866     442785
86867     501372
Name: building_id, Length: 86868, dtype: int64

In [47]:
submit = pd.merge(ids, damages, left_index=True,right_index=True).set_index('building_id')

In [48]:
submit

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,2
421793,3
...,...
310028,2
663567,2
1049160,2


In [49]:
submit.to_csv('submit_xgboost_v2.csv')